In [9]:
import sys
sys.path.append('../')

import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from utils.features.feature_extraction import Feature_extraction

from utils.constant import FEATURES

In [10]:
start = time.time()
pcapfiles = [
    '../inputs/sniffed.pcap',
]
destination_directory = '../outputs/'
n_threads = 8

address = "./"

for i in range(len(pcapfiles)):
    lstart = time.time()
    pcap_file = pcapfiles[i]
    fe = Feature_extraction()
    df = fe.pcap_evaluation(pcap_file,destination_directory + pcap_file)
    print(f'done! ({pcap_file})(' + str(round(time.time()-lstart, 2))+ 's)')
    
end = time.time()
print(f'Elapsed Time = {(end-start)}s')
df

done! (../inputs/sniffed.pcap)(0.5s)
Elapsed Time = 0.5011763572692871s


,ts,Protocol_name,Duration,Protocol Type,flow_duration,Header_Length,src_ip_bytes,fin_flag_number,syn_flag_number,rst_flag_number,...,Correlation,RARP,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time,source_ip,source_mac
0,1.701114e+09,TCP,48,6,0.000000,110,1,0,0,0,...,0.0,0,0,0,0,0,1.701114e+09,0.000000,35.169.123.237,0
1,1.701114e+09,TCP,128,6,0.044419,164,1,0,0,0,...,1.0,0,0,0,0,0,4.441905e-02,0.044419,10.216.22.121,0
2,1.701114e+09,TCP,128,6,0.000000,92,2,0,0,0,...,1.0,0,0,0,0,0,1.701114e+09,0.000000,10.216.22.121,0
3,1.701114e+09,TCP,240,6,0.025513,146,1,0,0,0,...,1.0,0,0,0,0,0,2.551293e-02,0.025513,34.238.229.83,0
4,1.701114e+09,UDP,64,17,0.000000,214,1,0,0,0,...,1.0,0,0,0,0,0,1.701114e+09,0.000000,10.216.22.123,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.701114e+09,TCP,64,6,0.000000,54,572,0,0,0,...,NaN,0,0,0,0,0,1.701114e+09,0.000000,10.216.22.110,0
996,1.701114e+09,TCP,64,6,0.000000,54,573,0,0,0,...,NaN,0,0,0,0,0,1.701114e+09,0.000000,10.216.22.110,0
997,1.701114e+09,TCP,64,6,0.000000,54,574,0,0,0,...,NaN,0,0,0,0,0,1.701114e+09,0.000000,10.216.22.110,0
998,1.701114e+09,TCP,64,6,0.000000,54,575,0,0,0,...,NaN,0,0,0,0,0,1.701114e+09,0.000000,10.216.22.110,0


In [14]:
# Preprocessing Functions
scaler = StandardScaler()
scaler.fit(df[FEATURES])
df[FEATURES] = scaler.transform(df[FEATURES])

df[df['source_ip'] == '10.216.22.110'][FEATURES]

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
423,-0.425257,-0.486391,-0.395931,-0.426925,-0.102451,-0.102451,0.0,-0.077693,-0.100504,-0.044766,...,-0.392486,-0.361105,-0.204762,-0.231821,-0.953821,-0.405731,-0.360281,-0.231597,1.151339,-1.038599
424,-0.425257,-0.486391,-0.395931,-0.426925,-0.102451,-0.102451,0.0,-0.077693,-0.100504,-0.044766,...,-0.408616,-0.368786,-0.204762,-0.231821,-0.780399,-0.437156,-0.367949,-0.231883,1.151339,-0.964147
425,-0.425257,-0.486391,-0.395931,-0.426925,-0.102451,-0.102451,0.0,-0.077693,-0.100504,-0.044766,...,-0.419369,-0.375143,-0.204762,-0.231821,-0.606977,-0.458448,-0.374296,-0.232102,1.151339,-0.873151
426,-0.425257,-0.486391,-0.395931,-0.426925,-0.102451,-0.102451,0.0,-0.077693,-0.100504,-0.044766,...,-0.427050,-0.380437,-0.204762,-0.231821,-0.433555,-0.473832,-0.379581,-0.232273,1.151339,-0.765610
427,-0.425257,-0.486391,-0.395931,-0.426925,-0.102451,-0.102451,0.0,-0.077693,-0.100504,-0.044766,...,-0.432811,-0.384906,-0.204762,-0.231821,-0.260133,-0.485468,-0.384043,-0.232409,1.151339,-0.641524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.425257,-0.486391,-0.395931,-0.426925,-0.102440,-0.102440,0.0,-0.077693,-0.100504,-0.044766,...,-0.473135,-0.461856,-0.204762,-0.231821,1.127243,-0.569459,-0.460866,-0.233546,-0.868554,0.946775
996,-0.425257,-0.486391,-0.395931,-0.426925,-0.102440,-0.102440,0.0,-0.077693,-0.100504,-0.044766,...,-0.473135,-0.461856,-0.204762,-0.231821,1.300665,-0.569459,-0.460866,-0.233546,-0.868554,1.219764
997,-0.425257,-0.486391,-0.395931,-0.426925,-0.102440,-0.102440,0.0,-0.077693,-0.100504,-0.044766,...,-0.473135,-0.461856,-0.204762,-0.231821,1.474087,-0.569459,-0.460866,-0.233546,-0.868554,1.509298
998,-0.425257,-0.486391,-0.395931,-0.426925,-0.102440,-0.102440,0.0,-0.077693,-0.100504,-0.044766,...,-0.473135,-0.461856,-0.204762,-0.231821,1.647509,-0.569459,-0.460866,-0.233546,-0.868554,1.815376


# Predict with our model

In [15]:
xgb_model = xgb.XGBClassifier()
from joblib import dump, load
xgb_model = load('../outputs/xgboost_10.joblib')

In [18]:
preds = xgb_model.predict(df[FEATURES])
import numpy as np
idx = np.where(preds == 1)[0]
idx_2 = np.where(df['source_ip']=='10.216.22.110')[0]
print(len(preds))
print(len(idx))
print(len(idx_2))
sum(preds)

1000
903
576


903